# PSO PySwarms

Parte 2: Implementar o mesmo problema usando a biblioteca pronta

In [ ]:
import numpy as np
import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def objective_function(x):
    if x.ndim == 1:
        x = x.reshape(1, -1)
    
    result = []
    for particle in x:
        fx_val = (particle[0] - 4) ** 2 - (particle[0] - 8) ** 3 + 5
        result.append(fx_val)
    
    return np.array(result)

initial_positions = np.array([[2.5], [5.6], [10.5]])

In [ ]:

options = {
    'c1': 0.7,  #coeficiente cognitivo 
    'c2': 0.2,  #coeficiente social
    'w': 0.3    #fator de inércia
}

#limites de busca
bounds = (np.array([0]), np.array([15]))  


n_particles = 3  
dimensions = 1   
max_iter = 10    


optimizer = ps.single.GlobalBestPSO(
    n_particles=n_particles, 
    dimensions=dimensions, 
    options=options, 
    bounds=bounds
)

In [ ]:
initial_positions = np.array([
    [2.5], 
    [5.6], 
    [10.5]
])

initial_velocities = np.zeros((n_particles, dimensions))

#config do otimizador
optimizer.reset()
optimizer.swarm.position = initial_positions.copy()
optimizer.swarm.velocity = initial_velocities.copy()

print("Posições iniciais das partículas:")
for i, pos in enumerate(initial_positions):
    print(f"Partícula {i}: {pos[0]}")

print("\nVelocidades iniciais das partículas:")
for i, vel in enumerate(initial_velocities):
    print(f"Partícula {i}: {vel[0]}")

In [ ]:
cost_history = []

for iteration in range(max_iter):
    print(f"-- Iteração {iteration + 1}")
    
    #fitness das posições atuais
    current_cost = objective_function(optimizer.swarm.position)
    cost_history.append(current_cost.copy())
    
    #fitness atual
    df_fitness = pd.DataFrame({
        'particle': range(n_particles),
        'fitness': current_cost
    })
    print("\nFitness:")
    print(df_fitness.to_string(index=False))
    
    #atualiza melhor pessoal/global
    optimizer.swarm._update_pbest()
    optimizer.swarm._update_gbest()
    
    #personal best
    pbest_data = []
    for i in range(n_particles):
        pbest_data.append({
            'particle': i,
            'x': optimizer.swarm.pbest_pos[i][0],
            'fit': optimizer.swarm.pbest_cost[i]
        })
    df_pbest = pd.DataFrame(pbest_data)
    print("\nPersonal Best (p_best):")
    print(df_pbest.to_string(index=False))
    
    #global best
    df_gbest = pd.DataFrame([{
        'x': optimizer.swarm.best_pos[0],
        'fit': optimizer.swarm.best_cost
    }])
    print("\nGlobal Best (g_best):")
    print(df_gbest.to_string(index=False))
    
    #novas velocidades (uma iteração do PSO)
    if iteration < max_iter - 1:  # não calcular na última iteração
        optimizer.swarm._update_velocity()
        
        df_velocity = pd.DataFrame({
            'particle': range(n_particles),
            'velocity': optimizer.swarm.velocity.flatten()
        })
        print("\nVelocities:")
        print(df_velocity.to_string(index=False))
        
        #atualiza posições
        optimizer.swarm._update_position()
        
        df_positions = pd.DataFrame({
            'particle': range(n_particles),
            'position': optimizer.swarm.position.flatten()
        })
        print("\nParticles (positions):")
        print(df_positions.to_string(index=False))
    
    print("\n" + "="*50 + "\n")

print("Resultados:")
print(f"Melhor posição encontrada: x = {optimizer.swarm.best_pos[0]:.6f}")
print(f"Melhor valor da função: f(x) = {optimizer.swarm.best_cost:.6f}")